In [2]:
import numpy as np
import pandas as pd
import matplotlib as mpl
from scipy.optimize import minimize
from matplotlib import pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')

plt.rcParams["font.family"] = "Malgun Gothic"
plt.rcParams["font.size"] = 15
plt.rcParams["figure.figsize"] = (8, 8)

# 마이너스 깨짐 해결
mpl.rcParams['axes.unicode_minus'] = False
# 디자인 설정
plt.style.use('fivethirtyeight')

In [3]:
path = r'C:\Users\owner\Desktop\study\4-1 전공수업\금융공학\금공중간 과제\IFE_P1\dataset.xlsx'
rtn_df = pd.read_excel(path)
rtn_df.set_index('Date',inplace=True)
rtn_df.head()

,SAMSUNG ELECTRONICS,HYUNDAI MOTOR,LG CHEM,NAVER,CELLTRION,LG HHLD.& HLTH.CARE,POSCO,SK TELECOM,AMOREPACIFIC,KB FINANCIAL GROUP,KOSPI,NASDAQ,SOX,KOR10Y
Date,,,,,,,,,,,,,,
2010-01-01,-0.023946,-0.055929,-0.122218,-0.093723,0.051464,-0.024349,-0.129325,0.082550,-0.142442,-0.165380,-0.048290,-0.066241,-0.141245,-0.011173
2010-02-01,-0.052368,0.000000,0.057432,0.048106,0.121623,-0.038874,0.000000,-0.053609,-0.026433,-0.050986,-0.005809,0.037532,0.065582,-0.040120
2010-03-01,0.072229,-0.008621,0.107442,-0.030092,0.092295,0.088064,-0.011300,-0.002878,0.054945,0.108214,0.048671,0.064844,0.070725,-0.037740
2010-04-01,0.027464,0.179406,0.171078,0.024555,0.084289,0.009950,-0.060271,-0.014430,0.018062,0.000000,0.026922,0.020321,0.016786,-0.022519
2010-05-01,-0.085184,0.025318,-0.028883,-0.061310,-0.013975,0.044879,-0.054625,-0.059365,0.084141,-0.100277,-0.055941,-0.091103,-0.061236,0.022519


In [16]:
stocks = rtn_df.columns[:-4] #10개 주식
markets = rtn_df.columns[-4:-1] #market port 
risk_free = rtn_df.columns[-1] #risk-free

#filtering
markets_df = rtn_df[markets]
stocks_df = rtn_df[stocks]
rf_df = pd.DataFrame(rtn_df[risk_free])

INDEX : KOSPI 

In [27]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

In [85]:
stocks_df.describe().iloc[1:3]

,SAMSUNG ELECTRONICS,HYUNDAI MOTOR,LG CHEM,NAVER,CELLTRION,LG HHLD.& HLTH.CARE,POSCO,SK TELECOM,AMOREPACIFIC,KB FINANCIAL GROUP
mean,0.006944,0.001003,0.001736,0.006960,0.016661,0.012305,-0.008112,0.002691,0.006542,-0.003323
std,0.066895,0.074286,0.092337,0.081385,0.140311,0.078979,0.066620,0.051756,0.102509,0.068665


#  Apply the CAPM for each stock based on market portfolio, KOSPI.


In [9]:
KOSPI = rtn_df['KOSPI'].values
KORY = rtn_df['KOR10Y'].values
for stock in stocks_df.columns:
    print('***********',stock,'***************')
    ri = rtn_df[stock].values
    cov = pd.DataFrame({'i':ri,'m':KOSPI}).cov()
    cov_i_m = cov.iloc[0,1]
    variance_m = cov.iloc[1,1]
    beta = cov_i_m/variance_m
    CAPM_ri_rf = beta * (np.mean(KOSPI) - np.mean(KORY))
    real_ri_rf = np.mean(ri) - np.mean(KORY)
    print('beta : {}'.format(beta))
    print('CAPM : {}'.format(CAPM_ri_rf))
    print('실제 : {}'.format(real_ri_rf))

*********** SAMSUNG ELECTRONICS ***************
beta : 1.0638254574007049
CAPM : 0.011756484276541216
실제 : 0.016714282937005033
*********** HYUNDAI MOTOR ***************
beta : 0.9855638426027241
CAPM : 0.010891604199242384
실제 : 0.010772956649827624
*********** LG CHEM ***************
beta : 1.3476860231340186
CAPM : 0.014893467185304973
실제 : 0.011505798526872916
*********** NAVER ***************
beta : 0.7888829881386075
CAPM : 0.008718056502185227
실제 : 0.016729347472333025
*********** CELLTRION ***************
beta : 0.6355352427192352
CAPM : 0.007023389068421389
실제 : 0.026430882737798714
*********** LG HHLD.& HLTH.CARE ***************
beta : 0.8065504241265059
CAPM : 0.008913301814236723
실제 : 0.022074813987989418
*********** POSCO ***************
beta : 0.9002135973087902
CAPM : 0.00994838667251693
실제 : 0.0016575470492826375
*********** SK TELECOM ***************
beta : 0.3201028826290772
CAPM : 0.003537501833899779
실제 : 0.012460801807984354
*********** AMOREPACIFIC ***************


# Apply the single-factor model for each stock based on diﬀerent market indices.


In [10]:
from sklearn.linear_model import LinearRegression

In [131]:
def analyze_single_factor():
    model = LinearRegression()
    residuals_dict = {}
    stocks_dict = {}
    for idx1, market in enumerate(markets_df.columns):
        f1 = markets_df[market].values
        residuals = np.zeros((len(stocks_df),len(stocks_df.columns)))

        stocks_dict[market] = {}
        tmp_stocks = pd.DataFrame(index = ['mean','var','cov','b','a','e-var','e-mean','e-factor corr']
                            ,columns = stocks_df.columns )
        for idx2, stock in enumerate(stocks_df.columns):
            info_list = []
            ri = stocks_df[stock].values
            model.fit(f1.reshape(-1,1),ri.reshape(-1,1))
            
            y_hat = model.predict(f1.reshape(-1,1))
            y_hats = []
            for i in y_hat:
                y_hats.extend(i)

            residual = ri - y_hats

            info_list.append(np.mean(ri))
            info_list.append(np.std(ri)**2)
            info_list.append(np.cov(ri,f1)[0,1])
            info_list.append(model.coef_[0][0])
            info_list.append(model.intercept_[0])
            info_list.append(round(np.std(residual)**2,7))
            info_list.append(round(np.mean(residual),7))
            info_list.append(round(np.corrcoef(residual, markets_df[market].values)[0,1],7))
            
            tmp_stocks[stock] = info_list # 정보 리스트 -> dataframe
            stocks_dict[market] = tmp_stocks # dataframe -> dic
            
            residuals[:,idx2] = residual


        residuals =  pd.DataFrame(residuals,columns = stocks_df.columns,index = stocks_df.index).corr(method='pearson')
        residuals_dict[market] = residuals
        
    return residuals_dict, stocks_dict

In [132]:
residual_dict, stocks_dict = analyze_single_factor()

In [133]:
residuals_dict.keys()

dict_keys(['KOSPI', 'NASDAQ', 'SOX'])

In [134]:
residuals_dict['KOSPI']

,SAMSUNG ELECTRONICS,HYUNDAI MOTOR,LG CHEM,NAVER,CELLTRION,LG HHLD.& HLTH.CARE,POSCO,SK TELECOM,AMOREPACIFIC,KB FINANCIAL GROUP
SAMSUNG ELECTRONICS,1.000000,-0.095019,-0.281820,0.044454,-0.306339,0.048110,-0.104567,-0.111717,-0.263558,-0.039652
HYUNDAI MOTOR,-0.095019,1.000000,0.047083,-0.124965,-0.048774,-0.084979,-0.087181,0.001029,-0.027829,-0.022866
LG CHEM,-0.281820,0.047083,1.000000,-0.006602,0.178427,-0.264088,0.044496,0.039935,-0.028418,0.029407
NAVER,0.044454,-0.124965,-0.006602,1.000000,-0.012645,0.122196,-0.100718,0.035361,0.105239,-0.005190
CELLTRION,-0.306339,-0.048774,0.178427,-0.012645,1.000000,0.075486,0.009791,-0.123711,0.232862,0.059185
LG HHLD.& HLTH.CARE,0.048110,-0.084979,-0.264088,0.122196,0.075486,1.000000,-0.233532,-0.200676,0.369648,-0.119538
POSCO,-0.104567,-0.087181,0.044496,-0.100718,0.009791,-0.233532,1.000000,0.041846,-0.221301,0.307851
SK TELECOM,-0.111717,0.001029,0.039935,0.035361,-0.123711,-0.200676,0.041846,1.000000,-0.097236,0.029814
AMOREPACIFIC,-0.263558,-0.027829,-0.028418,0.105239,0.232862,0.369648,-0.221301,-0.097236,1.000000,-0.142704
KB FINANCIAL GROUP,-0.039652,-0.022866,0.029407,-0.005190,0.059185,-0.119538,0.307851,0.029814,-0.142704,1.000000


In [135]:
residuals_dict['NASDAQ']

,SAMSUNG ELECTRONICS,HYUNDAI MOTOR,LG CHEM,NAVER,CELLTRION,LG HHLD.& HLTH.CARE,POSCO,SK TELECOM,AMOREPACIFIC,KB FINANCIAL GROUP
SAMSUNG ELECTRONICS,1.000000,0.184467,0.065084,0.147295,-0.190531,0.229500,0.112028,0.009033,0.015088,0.237406
HYUNDAI MOTOR,0.184467,1.000000,0.287540,0.024922,0.028556,0.104503,0.138574,0.100063,0.163544,0.269415
LG CHEM,0.065084,0.287540,1.000000,0.133091,0.226781,-0.020843,0.256974,0.140444,0.183208,0.325545
NAVER,0.147295,0.024922,0.133091,1.000000,0.023086,0.208691,-0.004349,0.085142,0.221558,0.112415
CELLTRION,-0.190531,0.028556,0.226781,0.023086,1.000000,0.122506,0.067808,-0.090319,0.273772,0.129916
LG HHLD.& HLTH.CARE,0.229500,0.104503,-0.020843,0.208691,0.122506,1.000000,-0.045379,-0.109116,0.460902,0.118071
POSCO,0.112028,0.138574,0.256974,-0.004349,0.067808,-0.045379,1.000000,0.121817,-0.001984,0.451125
SK TELECOM,0.009033,0.100063,0.140444,0.085142,-0.090319,-0.109116,0.121817,1.000000,-0.005669,0.139816
AMOREPACIFIC,0.015088,0.163544,0.183208,0.221558,0.273772,0.460902,-0.001984,-0.005669,1.000000,0.145373
KB FINANCIAL GROUP,0.237406,0.269415,0.325545,0.112415,0.129916,0.118071,0.451125,0.139816,0.145373,1.000000


In [140]:
residuals_dict['SOX']

,SAMSUNG ELECTRONICS,HYUNDAI MOTOR,LG CHEM,NAVER,CELLTRION,LG HHLD.& HLTH.CARE,POSCO,SK TELECOM,AMOREPACIFIC,KB FINANCIAL GROUP
SAMSUNG ELECTRONICS,1.000000,0.195891,0.080714,0.181034,-0.177877,0.278303,0.162714,0.039160,0.001294,0.294775
HYUNDAI MOTOR,0.195891,1.000000,0.289227,0.038497,0.030212,0.130618,0.156143,0.113039,0.154347,0.283455
LG CHEM,0.080714,0.289227,1.000000,0.146198,0.228037,0.012870,0.273785,0.155674,0.171463,0.340274
NAVER,0.181034,0.038497,0.146198,1.000000,0.029169,0.243879,0.039040,0.105508,0.210882,0.162550
CELLTRION,-0.177877,0.030212,0.228037,0.029169,1.000000,0.132552,0.075644,-0.083744,0.270910,0.135937
LG HHLD.& HLTH.CARE,0.278303,0.130618,0.012870,0.243879,0.132552,1.000000,0.008845,-0.086677,0.460636,0.180423
POSCO,0.162714,0.156143,0.273785,0.039040,0.075644,0.008845,1.000000,0.146002,-0.009213,0.492873
SK TELECOM,0.039160,0.113039,0.155674,0.105508,-0.083744,-0.086677,0.146002,1.000000,-0.004947,0.169910
AMOREPACIFIC,0.001294,0.154347,0.171463,0.210882,0.270910,0.460636,-0.009213,-0.004947,1.000000,0.127821
KB FINANCIAL GROUP,0.294775,0.283455,0.340274,0.162550,0.135937,0.180423,0.492873,0.169910,0.127821,1.000000


In [137]:
stocks_dict['KOSPI']

,SAMSUNG ELECTRONICS,HYUNDAI MOTOR,LG CHEM,NAVER,CELLTRION,LG HHLD.& HLTH.CARE,POSCO,SK TELECOM,AMOREPACIFIC,KB FINANCIAL GROUP
mean,0.006944,0.001003,0.001736,0.006960,0.016661,0.012305,-0.008112,0.002691,0.006542,-0.003323
var,0.004438,0.005472,0.008455,0.006568,0.019523,0.006186,0.004401,0.002656,0.010420,0.004676
cov,0.001693,0.001569,0.002145,0.001256,0.001012,0.001284,0.001433,0.000510,0.001553,0.001900
b,1.063825,0.985564,1.347686,0.788883,0.635535,0.806550,0.900214,0.320103,0.975768,1.193303
a,0.005581,-0.000260,0.000009,0.005949,0.015847,0.011272,-0.009266,0.002281,0.005291,-0.004852
e-var,0.002651,0.003939,0.005588,0.005586,0.018885,0.005159,0.003122,0.002495,0.008918,0.002428
e-mean,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000
e-factor corr,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,-0.000000,-0.000000,-0.000000,-0.000000


In [138]:
stocks_dict['NASDAQ']

,SAMSUNG ELECTRONICS,HYUNDAI MOTOR,LG CHEM,NAVER,CELLTRION,LG HHLD.& HLTH.CARE,POSCO,SK TELECOM,AMOREPACIFIC,KB FINANCIAL GROUP
mean,0.006944,0.001003,0.001736,0.006960,0.016661,0.012305,-0.008112,0.002691,0.006542,-0.003323
var,0.004438,0.005472,0.008455,0.006568,0.019523,0.006186,0.004401,0.002656,0.010420,0.004676
cov,0.001299,0.000790,0.001174,0.001160,0.000652,0.000678,0.001128,0.000312,0.000372,0.001511
b,0.721701,0.439088,0.652348,0.644219,0.362462,0.376850,0.626691,0.173080,0.206707,0.839384
a,0.000137,-0.003139,-0.004418,0.000883,0.013242,0.008750,-0.014024,0.001058,0.004592,-0.011241
e-var,0.003508,0.005128,0.007695,0.005828,0.019289,0.005932,0.003700,0.002603,0.010344,0.003418
e-mean,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000
e-factor corr,0.000000,-0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000


In [139]:
stocks_dict['SOX']

,SAMSUNG ELECTRONICS,HYUNDAI MOTOR,LG CHEM,NAVER,CELLTRION,LG HHLD.& HLTH.CARE,POSCO,SK TELECOM,AMOREPACIFIC,KB FINANCIAL GROUP
mean,0.006944,0.001003,0.001736,0.006960,0.016661,0.012305,-0.008112,0.002691,0.006542,-0.003323
var,0.004438,0.005472,0.008455,0.006568,0.019523,0.006186,0.004401,0.002656,0.010420,0.004676
cov,0.001651,0.001097,0.001661,0.001427,0.000892,0.000463,0.001312,0.000284,0.000775,0.001766
b,0.454198,0.301819,0.456879,0.392551,0.245446,0.127261,0.360878,0.078246,0.213304,0.485807
a,0.001901,-0.002348,-0.003337,0.002601,0.013936,0.010892,-0.012119,0.001822,0.004173,-0.008717
e-var,0.003694,0.005144,0.007703,0.006013,0.019306,0.006127,0.003932,0.002634,0.010256,0.003825
e-mean,0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,-0.000000
e-factor corr,0.000000,0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


# Apply the multi-factor model for each stock based on three market indices.


In [145]:
def analyze_multi_factor():
    model = LinearRegression()    
    ff = markets_df.values
    
    residuals = np.zeros((len(stocks_df),len(stocks_df.columns)))
    tmp_stocks = pd.DataFrame(index = ['mean','var','b-KOSPI','b-NASDA','b-SOX','a','e-var','e-mean','e-factor corr']
                        ,columns = stocks_df.columns )
    
    for idx2, stock in enumerate(stocks_df.columns):
        info_list = []
        ri = stocks_df[stock].values
        model.fit(ff,ri.reshape(-1,1))

        y_hat = model.predict(ff)
        y_hats = []
        for i in y_hat:
            y_hats.extend(i)

        residual = ri - y_hats

        info_list.append(np.mean(ri))
        info_list.append(np.std(ri)**2)
        #info_list.append(np.cov(ri,ff)[0,1])
        info_list.append(model.coef_[0][0])
        info_list.append(model.coef_[0][1])
        info_list.append(model.coef_[0][2])
        info_list.append(model.intercept_[0])
        info_list.append(round(np.std(residual)**2,7))
        info_list.append(round(np.mean(residual),7))
        info_list.append(round(np.corrcoef(residual, markets_df[market].values)[0,1],7))

        tmp_stocks[stock] = info_list # 정보 리스트 -> dataframe
        residuals[:,idx2] = residual


    residuals =  pd.DataFrame(residuals,columns = stocks_df.columns,index = stocks_df.index).corr(method='pearson')
        
    return residuals, tmp_stocks

In [146]:
residuals, stocks = analyze_multi_factor()

In [147]:
residuals

,SAMSUNG ELECTRONICS,HYUNDAI MOTOR,LG CHEM,NAVER,CELLTRION,LG HHLD.& HLTH.CARE,POSCO,SK TELECOM,AMOREPACIFIC,KB FINANCIAL GROUP
SAMSUNG ELECTRONICS,1.000000,-0.087508,-0.280046,0.034666,-0.307776,0.062959,-0.110472,-0.108070,-0.258512,-0.048871
HYUNDAI MOTOR,-0.087508,1.000000,0.024192,-0.110212,-0.052174,-0.088675,-0.074573,0.001969,-0.069629,-0.001136
LG CHEM,-0.280046,0.024192,1.000000,0.008186,0.177385,-0.266566,0.059028,0.043775,-0.069488,0.051638
NAVER,0.034666,-0.110212,0.008186,1.000000,-0.011678,0.136832,-0.111637,0.038828,0.137893,-0.021253
CELLTRION,-0.307776,-0.052174,0.177385,-0.011678,1.000000,0.079772,0.011456,-0.122855,0.235960,0.062225
LG HHLD.& HLTH.CARE,0.062959,-0.088675,-0.266566,0.136832,0.079772,1.000000,-0.238400,-0.217488,0.387206,-0.120712
POSCO,-0.110472,-0.074573,0.059028,-0.111637,0.011456,-0.238400,1.000000,0.041459,-0.206342,0.299492
SK TELECOM,-0.108070,0.001969,0.043775,0.038828,-0.122855,-0.217488,0.041459,1.000000,-0.099016,0.029764
AMOREPACIFIC,-0.258512,-0.069629,-0.069488,0.137893,0.235960,0.387206,-0.206342,-0.099016,1.000000,-0.112173
KB FINANCIAL GROUP,-0.048871,-0.001136,0.051638,-0.021253,0.062225,-0.120712,0.299492,0.029764,-0.112173,1.000000


In [148]:
stocks

,SAMSUNG ELECTRONICS,HYUNDAI MOTOR,LG CHEM,NAVER,CELLTRION,LG HHLD.& HLTH.CARE,POSCO,SK TELECOM,AMOREPACIFIC,KB FINANCIAL GROUP
mean,0.006944,0.001003,0.001736,0.006960,0.016661,0.012305,-0.008112,0.002691,0.006542,-0.003323
var,0.004438,0.005472,0.008455,0.006568,0.019523,0.006186,0.004401,0.002656,0.010420,0.004676
b-KOSPI,0.973957,1.171745,1.541170,0.594243,0.662397,0.939439,0.805670,0.345106,1.427497,1.057792
b-NASDA,0.051934,-0.437934,-0.569831,0.273076,-0.159990,0.283301,0.226820,0.089253,-1.069782,0.305162
b-SOX,0.064605,0.136008,0.235055,0.008735,0.098701,-0.389685,-0.072688,-0.102686,0.335901,-0.087894
a,0.004489,0.002123,0.002527,0.003525,0.016226,0.012756,-0.010477,0.002547,0.011074,-0.006581
e-var,0.002629,0.003845,0.005453,0.005502,0.018875,0.004978,0.003097,0.002483,0.008358,0.002380
e-mean,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000
e-factor corr,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000
